In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
import argparse
from pprint import pprint
from tools.runners.farm_runner import upgrade_farmv2_contracts, get_all_farm_v2_addresses, fetch_and_save_farms_from_chain,\
    pause_farm_contracts, resume_farm_contracts
from tools.runners.staking_runner import upgrade_staking_contracts, get_staking_addresses_from_chain,\
    pause_all_staking_contracts, resume_all_staking_contracts, fetch_and_save_stakings_from_chain
from tools.runners.metastaking_runner import upgrade_metastaking_v2_contracts, get_metastaking_v2_addresses,\
    fetch_and_save_metastakings_from_chain
from tools.runners.energy_factory_runner import upgrade_energy_factory, pause_energy_factory, resume_energy_factory
from tools.runner import fetch_and_save_pause_state
from utils.utils_chain import WrapperAddress, base64_to_hex
from contracts.farm_contract import FarmContract
from contracts.staking_contract import StakingContract
from contracts.metastaking_contract import MetaStakingContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

from context import Context

context = Context()

Move ownership of any lingering contract

Prep contract pause states

In [ ]:
fetch_and_save_farms_from_chain("")

In [ ]:
fetch_and_save_stakings_from_chain("")

In [ ]:
fetch_and_save_metastakings_from_chain("")

In [ ]:
fetch_and_save_pause_state("")

Pause contracts

In [ ]:
pause_farm_contracts("")

In [ ]:
pause_all_staking_contracts("")

In [ ]:
pause_energy_factory("")

Upgrade farm contracts

In [ ]:
farm_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-with-locked-rewards.wasm"
expected_code_hash = "c1f2a3652371510767f5d64a4df02a3eda2744d54e9d91f7680dbf1bd02d82bb"

args = argparse.Namespace(bytecode=farm_bytecode_path, compare_states=True)
upgrade_farmv2_contracts(args)

farm_addresses = get_all_farm_v2_addresses()
print(f"Checking {len(farm_addresses)} farm contracts for correct code hash...")
for address in farm_addresses:
    code_hash = context.network_provider.proxy.get_account(WrapperAddress(address)).code_hash
    assert base64_to_hex(code_hash) == expected_code_hash
print("Done!")

Upgrade staking contracts

In [ ]:
staking_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-staking.wasm"
expected_code_hash = "7eddcb67bef25c4a0c7de0c3e1b2444cbf84e05546ac61c29726b568f9efffde"

args = argparse.Namespace(bytecode=staking_bytecode_path, compare_states=True, all=True)
upgrade_staking_contracts(args)

staking_addresses = get_staking_addresses_from_chain()
print(f"Checking {len(staking_addresses)} staking contracts for correct code hash...")
for address in staking_addresses:
    code_hash = context.network_provider.proxy.get_account(WrapperAddress(address)).code_hash
    assert base64_to_hex(code_hash) == expected_code_hash
print("Done!")

Upgrade proxy staking contracts

In [ ]:
proxy_staking_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-staking-proxy.wasm"
expected_code_hash = "7eddcb67bef25c4a0c7de0c3e1b2444cbf84e05546ac61c29726b568f9efffde"

args = argparse.Namespace(bytecode=proxy_staking_bytecode_path, compare_states=True, all=True)
upgrade_metastaking_v2_contracts(args)

proxy_staking_addresses = get_metastaking_v2_addresses()
print(f"Checking {len(proxy_staking_addresses)} proxy-staking contracts for correct code hash...")
for address in proxy_staking_addresses:
    code_hash = context.network_provider.proxy.get_account(WrapperAddress(address)).code_hash
    assert base64_to_hex(code_hash) == expected_code_hash
print("Done!")

Add permission hub in contracts

In [ ]:
permissions_hub_contract = context.get_contracts(config.PERMISSIONS_HUBS)[0]
pprint(permissions_hub_contract.get_config_dict())

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
for address in farm_addresses:
    contract = FarmContract.load_contract_by_address(address)
    contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
for address in staking_addresses:
    contract = StakingContract.load_contract_by_address(address)
    contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
for address in proxy_staking_addresses:
    contract = MetaStakingContract.load_contract_by_address(address)
    contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)

Upgrade energy contract

In [ ]:
energy_factory_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/simple-lock-energy.wasm"
expected_code_hash = "c1f2a3652371510767f5d64a4df02a3eda2744d54e9d91f7680dbf1bd02d82bb"

args = argparse.Namespace(bytecode=energy_factory_bytecode_path, compare_states=True)
upgrade_energy_factory(args)

print(f"Checking energy contract for correct code hash...")
code_hash = context.network_provider.proxy.get_account(WrapperAddress(context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0].address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash
print("Done!")

Set multisig address for collection

In [ ]:
multisig_address = "erd1"

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
WrapperAddress(multisig_address)    # just to check if the address is valid
energy_contract.set_multisig_address(context.deployer_account, context.network_provider.proxy, multisig_address)

Add farms into unlocked whitelist

In [ ]:
farm_addresses = get_all_farm_v2_addresses()
energy_contract.add_sc_to_unlocked_token_mint_whitelist(context.deployer_account, context.network_provider.proxy, farm_addresses)

Resume contracts

In [ ]:
resume_energy_factory("")

In [ ]:
resume_farm_contracts("")

In [ ]:
resume_all_staking_contracts("")